---  
# Chocolate Milkshake fMRIPREP

In [1]:
# import pacakges 

import os, glob, shutil
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import subprocess as sp

import sys
sys.path.append("/projects/niblab/go_through/jupyter_notebooks")
import fMRIPreprocessing


from openpyxl import load_workbook
from IPython.display import SVG, display
from datetime import date

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', 159)
#pd.set_option('display.max_colwidth', -1)
from matplotlib import rcParams
rcParams['axes.titlepad'] = 15

%matplotlib inline 

In [2]:
# Variable load and setup
date = date.today()
study_folder_path="/projects/niblab/experiments/chocolate_milkshake"
data_folder_path= os.path.join(study_folder_path, "data")
bids_path = os.path.join(data_folder_path, "bids/bids_trimmed")
report_folder_path=os.path.join(data_folder_path, "quality_analysis")
fmriprep_path='/projects/niblab/experiments/chocolate_milkshake/data/fmriprep/fmriprep_trimmed'

In [3]:
fmriprep_folders=glob.glob(os.path.join(fmriprep_path, "sub-*/ses-*"))
fmriprep_ses1=[x.split("/")[-2] for x in fmriprep_folders if 'ses-1' in x]
fmriprep_ses2=[x.split("/")[-2] for x in fmriprep_folders if 'ses-2' in x]
fmriprep_ses3=[x.split("/")[-2] for x in fmriprep_folders if 'ses-3' in x]
fmriprep_ses4=[x.split("/")[-2] for x in fmriprep_folders if 'ses-4' in x]

runs=['run-1','run-2','run-3', 'run-4']
sessions=['ses-1','ses-2']
tasks=['task-training']

#subject_list=list(set(fmriprep_ses1))#+fmriprep_ses2))

#subject_list.sort()

bids_subs=[x.split("/")[-1] for x in glob.glob(os.path.join(data_folder_path,"bids/bids_trimmed/sub-*"))]


---  

## SDC Setup
We run this quick modification on BIDS before running FMRIPREP by adding functional references to the bids fmap (`.json`) files. This enables suscpetibility distortion correction (sdc) results from FMRIPREP.    
**If you get a `[INFO] can't edit file` error-**  
- check your bids folder permission and make sure you have writable permissions.  
    `chmod -R 775 {bids_data_folder}`

Code here: `/projects/niblab/jupyter_notebooks/fMRIPreprocessing.py`

In [10]:
# run sdc prep module
run_sdc=True #change to True to tcall the command
# as input the module requires a path to the bids data, containing sub-xx folders, and a session id
if run_sdc==True:
    sdc = fMRIPreprocessing.SDC(bids_path, "ses-4")
    sdc.fill_jsons()

[INFO] finished editing json files.


---

In [10]:
print("[INFO] %s fmriprep ses-1 subjects "%len(fmriprep_ses1))

[INFO] 0 fmriprep ses-1 subjects 


In [11]:
print("[INFO] %s bids subjects "%len(bids_subs))

[INFO] 135 bids subjects 


In [14]:
bids_subs.sort()
bids_subs[1:3]

['sub-002', 'sub-004']

In [9]:
def submit_fmriprep_batch(job_file, x, y, n, submit_job=False):
    print('[INFO] batch file: %s'%job_file)
    batch_cmd='sbatch --array={}-{}%{} {}'.format(x,y,n,job_file)
    print('[INFO] batch command: {}'.format(batch_cmd))
    
    # submit batch job
    if submit_job==True: 
        sp.run(batch_cmd, shell=True)        
        print('[INFO] submitted job.')

In [10]:
# Set Parameters 
fmriprep_job_file='/projects/niblab/experiments/chocolate_milkshake/code/fmriprep.job'

x=1
y=155
n=135
submit_fmriprep_batch(fmriprep_job_file,x,y, n, submit_job=True)

[INFO] batch file: /projects/niblab/experiments/chocolate_milkshake/code/fmriprep.job
[INFO] batch command: sbatch --array=1-155%135 /projects/niblab/experiments/chocolate_milkshake/code/fmriprep.job
[INFO] submitted job.


In [ ]:
# Set Parameters 
fmriprep_job_file='/projects/niblab/experiments/chocolate_milkshake/code/fmriprep.job'




submit_fmriprep_batch(fmriprep_job_file,num_id,num_id, n=1, submit_job)

submission_list=bids_subs[1:]



# run single submission
for sub_id in submission_list:
    num_id=sub_id.split("-")[1].lstrip('0')
    submit_job=False # set to True when you want to run the file 
    submit_fmriprep_batch(fmriprep_job_file,num_id,num_id, n=1, submit_job)

---  
## Quality Report  

In [34]:
"""
# Methods
"""

def quality_report(bids_path,fmriprep_path, sessions):
    
    sessions_missing={}
    sessions_found={}
    for ses in sessions: 

        bids_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(bids_path, "sub-*/%s"%ses))]

        bids_subject_list=[x for x in bids_subject_list ]#if x not in remove_subs]

        fmriprep_folders=glob.glob(os.path.join(fmriprep_path, "sub-*/ses-*"))

        fprep_subject_list=[x.split("/")[-2] for x in fmriprep_folders if ses in x]

        fprep_missing = [x for x in bids_subject_list if x not in fprep_subject_list]
        
        sessions_found[ses]=fprep_subject_list
        sessions_missing[ses]=fprep_missing
    
    missing_list=[]
    for x in sessions_missing.values():
        for y in x:
            missing_list.append(y)
            
    missing_list=list(set(missing_list))  
    missing_list.sort()
    print('[INFO] missing subjects: ', missing_list)

    for ses in sessions_found:
        print("\n[INFO] %s subjects found (%s total): \n %s \n"%(ses, len(sessions_found[ses]), sessions_found[ses]))



    
"""
# Build FMRIPREP Report 

"""


def build_fmriprep_report(fmriprep_path, sessions, tasks, sub_ct, write_files=False):
    
    print('[INFO] fmriprep data path: %s '%fmriprep_path)
    dataframes=[]
    df_dict={}
    
    for session in sessions:
        
        data_dict={} #initialize data dictionary for session
        
        for i in range(1,sub_ct+1):
           
            #milk_ct=0
            
        
            subject="sub-%s"%f'{i:03}'
            bids_folder=os.path.join(fmriprep_path, subject,
                                    session)

            if subject not in data_dict:
                data_dict[subject] = {}

            # get anat file and save plot
            #_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g
            anat_imgs =glob.glob(os.path.join(fmriprep_path, subject, "anat",
                   '%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'%(subject)))

            #if os.path.exists(anat_img): 
            data_dict[subject]["anat_file_ct"]=len(anat_imgs)

            # get functional files and check their volume and plot images
            func_files=glob.glob(os.path.join(
                    fmriprep_path, subject, session,
                    "func/*task-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" ))
            data_dict[subject]["task_file_ct"]=len(func_files)
            
            milk_ids=[]
            milk_ct=0
            
            for func_file in func_files:
                
                task=func_file.split("/")[-1].split("_")[2]
                vol = sp.check_output(["fslnvols", func_file])
                vol=str(vol,'utf-8').strip("\n")
                

                if "milkshake"  in task:
                    milk_ct+=1
                    milk_id=task.split("-")[1][9]
                    milk_ids.append(milk_id)
                    
                    var_name="%s_vol"%("milkshake%s"%str(milk_ct))
                else:
                    var_name="%s_vol"%task
                    
                data_dict[subject][var_name]=vol
          
            data_dict[subject]["milkshake_ids"]=milk_ids
        
        dataframe=pd.DataFrame(data_dict).T        
        dataframes.append((dataframe, session))
        #dataframe.to_excel(writer, sheet_name="%s_fmriprep"%session)
    #writer.save()
    #writer.close()
    return dataframes;


In [35]:
bids_path="/projects/niblab/experiments/chocolate_milkshake/data/bids/bids_trimmed"
fmriprep_path="/projects/niblab/experiments/chocolate_milkshake/data/fmriprep_trimmed/fmriprep"
sessions=['ses-1', 'ses-2', 'ses-3', 'ses-4']

In [36]:
dfs=build_fmriprep_report(fmriprep_path, sessions, tasks, sub_ct=140, write_files=False)

[INFO] fmriprep data path: /projects/niblab/experiments/chocolate_milkshake/data/fmriprep_trimmed/fmriprep 


In [38]:
df1=dfs[0][0]
print('[INFO] session: %s'%dfs[0][1])
df1.head()

[INFO] session: ses-1


,anat_file_ct,task_file_ct,milkshake1_vol,milkshake2_vol,task-imagine_vol,milkshake_ids
sub-001,1,3,441,441,475,"[C, B]"
sub-002,1,3,443,415,475,"[B, A]"
sub-003,0,0,NaN,NaN,NaN,[]
sub-004,1,3,441,441,475,"[B, A]"
sub-005,1,3,441,441,475,"[C, B]"


In [39]:
df1=dfs[1][0]
print('[INFO] session: %s'%dfs[1][1])
df1.head()

[INFO] session: ses-2


,anat_file_ct,task_file_ct,milkshake1_vol,milkshake2_vol,task-imagine_vol,milkshake_ids
sub-001,1,3,441,441,475,"[C, B]"
sub-002,1,3,441,441,475,"[B, A]"
sub-003,0,0,NaN,NaN,NaN,[]
sub-004,1,3,441,441,475,"[C, B]"
sub-005,1,3,441,441,475,"[D, B]"


In [40]:
df1=dfs[2][0]
print('[INFO] session: %s'%dfs[2][1])
df1.head()

[INFO] session: ses-3


,anat_file_ct,task_file_ct,milkshake1_vol,milkshake2_vol,task-imagine_vol,milkshake_ids
sub-001,1,3,441,441,475,"[D, C]"
sub-002,1,3,441,441,475,"[D, B]"
sub-003,0,0,NaN,NaN,NaN,[]
sub-004,1,3,441,441,475,"[D, C]"
sub-005,1,3,441,441,475,"[B, A]"


In [42]:
df4=dfs[3][0]
print('[INFO] session: %s'%dfs[3][1])
df4.head()

[INFO] session: ses-4


,anat_file_ct,task_file_ct,milkshake1_vol,milkshake2_vol,task-imagine_vol,milkshake_ids
sub-001,1,3,441,441,475,"[D, A]"
sub-002,1,3,441,441,475,"[C, A]"
sub-003,0,0,NaN,NaN,NaN,[]
sub-004,1,3,441,441,475,"[D, C]"
sub-005,1,3,441,441,475,"[D, B]"


In [7]:
quality_report(bids_path,fmriprep_path, sessions)

[INFO] missing subjects:  ['sub-118', 'sub-125', 'sub-127', 'sub-137', 'sub-138', 'sub-139', 'sub-142', 'sub-143', 'sub-144', 'sub-145', 'sub-146', 'sub-147', 'sub-150', 'sub-151', 'sub-152', 'sub-153', 'sub-154']

[INFO] ses-1 subjects found (116 total): 
 ['sub-001', 'sub-002', 'sub-004', 'sub-006', 'sub-005', 'sub-015', 'sub-020', 'sub-021', 'sub-019', 'sub-010', 'sub-013', 'sub-014', 'sub-011', 'sub-022', 'sub-017', 'sub-012', 'sub-018', 'sub-008', 'sub-009', 'sub-026', 'sub-027', 'sub-029', 'sub-030', 'sub-028', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-047', 'sub-046', 'sub-048', 'sub-049', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-057', 'sub-056', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-068', 'sub-069', 'sub-071', 'sub-072', 'sub-073', 'sub-075', 'sub-077', 'sub-076', 'sub-080', 'sub-079', 'sub

---